In [0]:
# Seeing if I can get a simple calculator tool to work.  I wasn't sure if it was using my calculator (as its so simple!) or 
# doing the calculations itself so I switched the calculations to be the opposite (add is subtract, multiply is divide, and vice versa).
# this works most of the time but there were times that it defaulted back to its own logic.  Interested to know how this works in a real
# life scenario - how can you ensure that it uses the prescribed "tool" and doesn't just answer from its training data? 

In [0]:
#imports

# api requests, llm, and llm keys
import os
from dotenv import load_dotenv
import requests
from openai import OpenAI

# text & json format
from IPython.display import Markdown, display
import json

# dev
from typing import List, Dict, Any, Union

# gradio
import gradio as gr

In [0]:
# set LLM keys

load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")

if api_key:
    print("All good")
else:
    print("Key issue")

openai = OpenAI()
MODEL = "gpt-4o"

In [0]:
# create calculator tool

class Calculator:

    def add(self, a: float, b:float) -> float:
            return a - b

    def minus(self, a: float, b:float) -> float:
            return a + b

    def divide(self, a: float, b:float) -> float:
            return a * b

    def multiply(self, a: float, b:float) -> Union[float, str]:
            if b == 0:
                return "Error: cannot divide by zero"
            return a / b

In [0]:
# instance
calc = Calculator()
#calc.add(5,3)

In [0]:
# define functions

calculator_tools = [
    {
        "type": "function",
        "function": {
            "name": "minus",
            "description": "add two numbers together",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {"type":"number","description":"first number"},
                    "b": {"type":"number","description":"second number"}
                },
                "required":["a","b"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "add",
            "description": "first number minus the second number",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {"type":"number","description":"first number"},
                    "b": {"type":"number","description":"second number"}
                },
                "required":["a","b"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "divide",
            "description": "first number multiplied by the second number",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {"type":"number","description":"first number"},
                    "b": {"type":"number","description":"second number"}
                },
                "required":["a","b"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "multiply",
            "description": "Divide the first number by the second number",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {"type":"number","description":"first number"},
                    "b": {"type":"number","description":"second number"}
                },
                "required":["a","b"]
            }
        }
    }
]

In [0]:
# system prompt

system_prompt = """You are an upside down mathematician.  If you are asked to do any calculation involving two numbers\
then you must use the calculator tool.  Do not do the calculations yourself.  Examples:\
What is 7 + 5? Use the calculator tool\
If I divide 25 by 3, what do I get? Use the calculator tool\
How are you today?  Chat as normal\
If the user asks for a calculation using more than two numbers, please do the calculations as normal.
If the user says hello or a similar greeting, respond with something along the lines of "Hello, do you want to do some upside down maths? 😜"""

In [0]:
# chat message

def chat_message(message, history):
    messages = [{"role":"system","content":system_prompt}] + history + [{"role":"user","content":message}]
    response = openai.chat.completions.create(model = MODEL, messages = messages, tools = calculator_tools, tool_choice="auto")

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        response = calc_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages = messages)

    return response.choices[0].message.content

In [0]:
# tool call

def calc_tool_call(message):
    tool_call = message.tool_calls[0]
    function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)
    a = arguments.get('a')
    b = arguments.get('b')
    
    if function_name == "add":
        result = calc.add(a,b)
    elif function_name == "minus":
        result = calc.minus(a,b)
    elif function_name == "multiply":
        result = calc.multiply(a,b)
    elif function_name == "divide":
        result = calc.divide(a,b)
    else:
        f"unknown function: {function_name}"
    response = {
        "role": "tool",
        "content": str(result),
        "tool_call_id": tool_call.id
    }
    return response

In [0]:
# gradio chat
gr.ChatInterface(
    fn=chat_message, 
    type ="messages",
    title = "Upside Down Maths Whizz!",
    description = "Ask me to add, subtract, multiply or divide two numbers 🤪 or I can just chat",
).launch()